In [25]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/datos'
CP_FILE = 'Canarias_CP.csv'
FOLDER_BBVA = 'CM01_3'

In [45]:
from pathlib import Path
from pandas import read_csv, DataFrame

class BbvaData:
    def __init__(self, base_dir:str, cp_file:str, folder_name:str) -> None:
        self.base_dir = Path(base_dir)
        self.cp_filepath = self.base_dir.joinpath(cp_file)
        self.folder_path = self.base_dir.joinpath(folder_name)
        self.avg_am_files = self.folder_path.glob('avg_am*.csv')
        self.cards_files = self.folder_path.glob('cards*.csv')

    def __len__(self) -> tuple:
        return (len(list(self.avg_am_files)),len(list(self.cards_files)))

    def __str__(self) -> str:
        len_files = self.__len__()
        return f'Folder path:\t{self.folder_path}\nWith {len_files[0]} files about average amount and {len_files[-1]} about cards'

    def filter_zipcodes(self, country_col='country', country='ES', zipcode_col='zipcode') -> DataFrame:
        zipcode = read_csv(self.cp_filepath)
        valid_zipcodes = set(zipcode[zipcode_col])
        avg_am = DataFrame()
        cards = DataFrame()
        for avg_am_path, cards_path in zip(self.avg_am_files,self.cards_files):
            avg_am_sub = read_csv(avg_am_path)
            avg_am_sub = avg_am_sub[avg_am_sub[country_col]==country].copy()
            avg_am_sub = avg_am_sub[avg_am_sub[zipcode_col].isin(valid_zipcodes)].copy()
            avg_am = avg_am.append(avg_am_sub, ignore_index=True)

            cards_sub = read_csv(cards_path)
            cards_sub = cards_sub[cards_sub[country_col]==country].copy()
            cards_sub = cards_sub[cards_sub[zipcode_col].isin(valid_zipcodes)].copy()
            cards = cards.append(cards_sub, ignore_index=True)

        df = avg_am.merge(cards)
        return df

    def create



bbva = BbvaData(BASE_DIR, CP_FILE, FOLDER_BBVA)
bbva.filter_zipcodes()

,zipcode,country,day,avg_amount,cards
0,35004,ES,2018-09-01,24.88,26.0
1,35004,ES,2018-09-02,23.83,5.0
2,35004,ES,2018-09-03,19.82,33.0
3,35004,ES,2018-09-04,13.26,19.0
4,35004,ES,2018-09-05,34.40,10.0
...,...,...,...,...,...
73826,35009,ES,2020-08-27,22.33,16.0
73827,35009,ES,2020-08-28,25.53,17.0
73828,35009,ES,2020-08-29,27.75,8.0
73829,35009,ES,2020-08-30,34.05,3.0
